In [11]:
import time
import pandas as pd
from tqdm.notebook import tqdm
import random
import networkx as nx
from IPython.display import display
from ipywidgets import IntProgress

In [12]:
def generate_graph(n, restricted_edge=0.1, distance_range=(1, 100), time_range=(1, 60)):
    # Crée un graphe complet avec n sommets
    G = nx.complete_graph(n)

    # Parcourt toutes les arêtes du graphe
    for u, v in G.edges():
        # Pour chaque arête, on attribue une distance et un temps
        # Avec une probabilité 'restricted_edge', on met une valeur élevée (999) pour simuler une contrainte
        G[u][v]['distance'] = 999 if random.random() < restricted_edge else random.randint(*distance_range)
        G[u][v]['time'] = 999 if random.random() < restricted_edge else random.randint(*time_range)

    return G

In [17]:
def ant_colony_vrp_fast(graph, depot, nombre_camion=2, num_ants=5, iterations=300,
                        alpha=2, beta=4, evaporation_rate=0.5, pheromone_init=1.5):
    # Initialisation des phéromones sur chaque arête
    pheromones = {edge: pheromone_init for edge in graph.edges}
    
    best_routes = None
    best_cost = float('inf')
    
    # Liste des nœuds sans le dépôt
    nodes = list(graph.nodes)
    nodes.remove(depot)
    
    # Répartition approximative des clients par véhicule
    nodes_per_vehicle = len(nodes) // nombre_camion
    extra_nodes = len(nodes) % nombre_camion
    
    courantes = []              # Coût minimal trouvé à chaque itération
    meilleures_courantes = []   # Meilleur coût global trouvé jusqu'à chaque itération

    for iteration in range(iterations):
        iteration_best_cost = float('inf')  # Meilleur coût de cette itération

        for ant in range(num_ants):  # Pour chaque fourmi
            unvisited = set(graph.nodes)
            unvisited.remove(depot)
            routes = []
            total_cost = 0

            for vehicle in range(nombre_camion):
                current_route = [depot]
                current_node = depot
                visited_this_trip = set()
                
                # Nombre de clients à visiter pour ce véhicule
                nodes_to_visit = nodes_per_vehicle + (1 if vehicle < extra_nodes else 0)

                while unvisited and len(visited_this_trip) < nodes_to_visit:
                    neighbors = [n for n in graph.neighbors(current_node) if n in unvisited]
                    if not neighbors:
                        break

                    probabilities = []
                    for neighbor in neighbors:
                        # Calcul de la probabilité en fonction des phéromones et de la distance
                        edge = (current_node, neighbor) if (current_node, neighbor) in pheromones else (neighbor, current_node)
                        pheromone = pheromones[edge]
                        distance = graph[current_node][neighbor]['distance']
                        probabilities.append((neighbor, (pheromone ** alpha) * ((1 / distance) ** beta)))

                    if not probabilities:
                        break

                    total_prob = sum(prob[1] for prob in probabilities)
                    # Choix du prochain nœud en fonction des probabilités
                    next_node = random.choices(
                        [node for node, _ in probabilities],
                        [prob / total_prob for _, prob in probabilities]
                    )[0]

                    current_route.append(next_node)
                    total_cost += graph[current_node][next_node]['distance']
                    unvisited.remove(next_node)
                    visited_this_trip.add(next_node)
                    current_node = next_node

                # Retour au dépôt à la fin de la tournée
                if current_node != depot:
                    current_route.append(depot)
                    total_cost += graph[current_node][depot]['distance']

                routes.append(current_route)

            # Mise à jour du meilleur coût global et local
            if total_cost < best_cost:
                best_cost = total_cost
                best_routes = routes

            if total_cost < iteration_best_cost:
                iteration_best_cost = total_cost

        courantes.append(iteration_best_cost)
        meilleures_courantes.append(best_cost)

        # Évaporation des phéromones
        for edge in pheromones:
            pheromones[edge] *= (1 - evaporation_rate)

        # Renforcement des arêtes empruntées dans la meilleure solution trouvée
        for route in best_routes:
            for i in range(len(route) - 1):
                edge = (route[i], route[i + 1]) if (route[i], route[i + 1]) in pheromones else (route[i + 1], route[i])
                pheromones[edge] += 1 / best_cost

    # Conversion des routes en listes d'arêtes pour affichage
    formatted_routes = []
    for route in best_routes:
        edges = [(route[i], route[i + 1]) for i in range(len(route) - 1)]
        formatted_routes.append(edges)

    return formatted_routes, best_cost


In [18]:


seed = random.randint(0, 100000)

# Initialise le générateur de nombres aléatoires avec cette graine
# Cela permet de reproduire exactement les mêmes résultats si on relance le programme avec la même seed
random.seed(seed)


# Paramètres spécifiques à tester
graph_sizes = [5, 10, 15, 20, 30]
num_ants = 5
num_iterations = 200  # ou ce que tu veux pour tester
alpha = 2
beta_list = [2, 3, 4, 5]
evaporation_rate = 0.5
pheromone_init_list = [1.0, 1.5, 2.0]

# Pour stocker les résultats
results_precise = []
execution_times_precise = []

# Barre de progression
nb_tests = len(graph_sizes) * len(beta_list) * len(pheromone_init_list)
bar = IntProgress(min=0, max=nb_tests, layout={"width": "100%"})
display(bar)

# Boucle de calcul
for size in tqdm(graph_sizes, desc="Graph Sizes"):
    G = generate_graph(size, restricted_edge=0.1, distance_range=(1, 100), time_range=(1, 60))

    
    for beta in beta_list:
        for pheromone_init in pheromone_init_list:
            start_time = time.time()
            best_routes, best_cost = ant_colony_vrp_fast(
                G, depot=0,
                num_ants=num_ants,
                alpha=alpha,
                beta=beta,
                evaporation_rate=evaporation_rate,
                pheromone_init=pheromone_init
            )
            end_time = time.time()
            exec_time = end_time - start_time

            results_precise.append((
                size, num_ants, num_iterations, alpha, beta, evaporation_rate, pheromone_init, best_cost
            ))
            execution_times_precise.append((
                size, num_ants, num_iterations, alpha, beta, evaporation_rate, pheromone_init, exec_time
            ))
            bar.value += 1

bar.close()


IntProgress(value=0, layout=Layout(width='100%'), max=60)

Graph Sizes:   0%|          | 0/5 [00:00<?, ?it/s]

In [19]:
# Conversion en DataFrames
results_precise_df = pd.DataFrame(results_precise, columns=[
    "Graph Size", "Num Ants", "Num Iterations", "Alpha", "Beta", "Evaporation Rate", "Pheromone Init", "Best Cost"
])
execution_times_precise_df = pd.DataFrame(execution_times_precise, columns=[
    "Graph Size", "Num Ants", "Num Iterations", "Alpha", "Beta", "Evaporation Rate", "Pheromone Init", "Execution Time"
])


In [22]:
# Fusion des deux
merged_precise = pd.merge(
    results_precise_df.drop(columns=["Num Iterations"]),
    execution_times_precise_df.drop(columns=["Num Iterations"]),
    on=["Graph Size", "Num Ants", "Alpha", "Beta", "Evaporation Rate", "Pheromone Init"]
)

# Calcul des moyennes et écarts-types
summary_precise = merged_precise.groupby(["Graph Size", "Beta", "Pheromone Init"]).agg({
    "Best Cost": "mean",
    "Execution Time": ["mean", "std"]
}).reset_index()

# Renommage des colonnes
summary_precise.columns = [
    "Graph Size", "Beta", "Pheromone Init",
    "Mean Best Cost", "Mean Execution Time", "Std Execution Time"
]
# Affichage en tableau séparé par taille, trié par Best Cost croissant
for size in sorted(summary_precise["Graph Size"].unique()):
    print(f"\n=== Taille de graphe: {size} ===")
    summary_sorted = summary_precise[summary_precise["Graph Size"] == size]\
        .sort_values(by="Mean Best Cost", ascending=True)
    display(summary_sorted)


=== Taille de graphe: 5 ===


,Graph Size,Beta,Pheromone Init,Mean Best Cost,Mean Execution Time,Std Execution Time
0,5,2,1.0,180.0,0.070735,NaN
1,5,2,1.5,180.0,0.062240,NaN
2,5,2,2.0,180.0,0.056479,NaN
3,5,3,1.0,180.0,0.057215,NaN
4,5,3,1.5,180.0,0.053882,NaN
5,5,3,2.0,180.0,0.061900,NaN
6,5,4,1.0,180.0,0.069738,NaN
7,5,4,1.5,180.0,0.060426,NaN
8,5,4,2.0,180.0,0.055524,NaN
9,5,5,1.0,180.0,0.061324,NaN



=== Taille de graphe: 10 ===


,Graph Size,Beta,Pheromone Init,Mean Best Cost,Mean Execution Time,Std Execution Time
16,10,3,1.5,279.0,0.196166,NaN
12,10,2,1.0,305.0,0.205659,NaN
13,10,2,1.5,305.0,0.212409,NaN
20,10,4,2.0,306.0,0.184601,NaN
14,10,2,2.0,318.0,0.204832,NaN
15,10,3,1.0,318.0,0.198948,NaN
17,10,3,2.0,323.0,0.201585,NaN
18,10,4,1.0,335.0,0.186376,NaN
19,10,4,1.5,335.0,0.206897,NaN
21,10,5,1.0,335.0,0.217064,NaN



=== Taille de graphe: 15 ===


,Graph Size,Beta,Pheromone Init,Mean Best Cost,Mean Execution Time,Std Execution Time
25,15,2,1.5,268.0,0.369033,NaN
27,15,3,1.0,268.0,0.376244,NaN
28,15,3,1.5,268.0,0.410676,NaN
29,15,3,2.0,268.0,0.405180,NaN
30,15,4,1.0,268.0,0.388860,NaN
31,15,4,1.5,268.0,0.396499,NaN
32,15,4,2.0,268.0,0.399617,NaN
33,15,5,1.0,268.0,0.383554,NaN
34,15,5,1.5,268.0,0.366987,NaN
35,15,5,2.0,268.0,0.371427,NaN



=== Taille de graphe: 20 ===


,Graph Size,Beta,Pheromone Init,Mean Best Cost,Mean Execution Time,Std Execution Time
47,20,5,2.0,334.0,0.646117,NaN
41,20,3,2.0,351.0,0.688771,NaN
39,20,3,1.0,358.0,0.688811,NaN
36,20,2,1.0,374.0,0.660641,NaN
37,20,2,1.5,378.0,0.620322,NaN
40,20,3,1.5,379.0,0.654626,NaN
42,20,4,1.0,381.0,0.663506,NaN
43,20,4,1.5,392.0,0.704175,NaN
46,20,5,1.5,401.0,0.670532,NaN
38,20,2,2.0,415.0,0.690588,NaN



=== Taille de graphe: 30 ===


,Graph Size,Beta,Pheromone Init,Mean Best Cost,Mean Execution Time,Std Execution Time
48,30,2,1.0,357.0,1.591460,NaN
58,30,5,1.5,370.0,2.271230,NaN
51,30,3,1.0,373.0,1.539644,NaN
56,30,4,2.0,380.0,1.999918,NaN
57,30,5,1.0,383.0,2.118283,NaN
52,30,3,1.5,385.0,1.389961,NaN
53,30,3,2.0,391.0,1.348477,NaN
55,30,4,1.5,395.0,1.590338,NaN
59,30,5,2.0,401.0,1.624375,NaN
54,30,4,1.0,420.0,1.402355,NaN
